In [17]:
# Cell 1: Import libraries
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [18]:
# Cell 2: Load tokenizer and model (lightweight)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = TFAutoModel.from_pretrained(model_name)
bert_model.trainable = False  # freeze BERT for speedame)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [19]:
# Cell 3: Load your dataset (adjust path as needed)
# Example CSV expected with 'text' and 'label' columns
df = pd.read_csv("datasets/labelled.csv") 

In [20]:
# Combine 'Heading' and 'Body' into a single text column
df["text"] = df["Heading"].fillna("") + " " + df["Body"].fillna("")

# Encode labels
le = LabelEncoder()
df["label"] = le.fit_transform(df["Category"])

# Tokenize text
tokens = tokenizer(
    df["text"].tolist(),
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors="tf"
)

# Convert to numpy for sklearn compatibility
input_ids_np = tokens["input_ids"].numpy()
attention_mask_np = tokens["attention_mask"].numpy()
labels_np = df["label"].values

# Train-test split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
    input_ids_np, attention_mask_np, labels_np, test_size=0.2, random_state=42
)

# Create TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices(
    ((X_train_ids, X_train_mask), y_train)
).batch(8)

test_dataset = tf.data.Dataset.from_tensor_slices(
    ((X_test_ids, X_test_mask), y_test)
).batch(8)

In [21]:
# Cell 4: Build classifier model
input_ids_layer = tf.keras.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask_layer = tf.keras.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

embedding = bert_model(input_ids=input_ids_layer, attention_mask=attention_mask_layer)[0][:, 0, :]  # CLS token
x = tf.keras.layers.Dense(128, activation='relu')(embedding)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(x)

model = tf.keras.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=output)

In [22]:
# Cell 5: Compile and train the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
model.fit(train_dataset, validation_data=test_dataset, epochs=2)

Epoch 1/2
1159/1159 [==============================] - 2498s 2s/step - loss: 0.9796 - accuracy: 0.6768 - val_loss: 0.7863 - val_accuracy: 0.7203
Epoch 2/2
1159/1159 [==============================] - 2092s 2s/step - loss: 0.7789 - accuracy: 0.7419 - val_loss: 0.7504 - val_accuracy: 0.7385


In [25]:
model.save("bert_text_classifier_model.h5")

